## Preparation

In [2]:
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import pandas as pd
import sklearn
import os
import sys
import time
import tensorflow as tf
from tensorflow import keras

print(tf.__version__)
print(sys.version_info)
for module in mpl, np, pd, sklearn, tf, keras:
    print(module.__name__, module.__version__)

2.0.0
sys.version_info(major=3, minor=7, micro=9, releaselevel='final', serial=0)
matplotlib 3.3.3
numpy 1.18.4
pandas 1.2.0
sklearn 0.24.0
tensorflow 2.0.0
tensorflow_core.keras 2.2.4-tf


In [ ]:
from sklearn.datasets import fetch_california_housing
housing = fetch_california_housing()
# print(housing.DESCR)
print(housing.data.shape)
print(housing.target.shape)

In [3]:
from sklearn.model_selection import train_test_split
xtrain_all, xtest, ytrain_all, ytest = train_test_split(housing.data, housing.target, random_state=7)
xtrain, xvalid, ytrain, yvalid = train_test_split(xtrain_all ,ytrain_all, random_state=11)

print(xtrain.shape, ytrain.shape)
print(xvalid.shape, yvalid.shape)
print(xtest.shape, ytest.shape)

(11610, 8) (11610,)
(3870, 8) (3870,)
(5160, 8) (5160,)


## Normalization

In [4]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
xtrain_scaled = scaler.fit_transform(xtrain)
xvalid_scaled= scaler.transform(xvalid)
xtest_scaled = scaler.transform(xtest)

## Modeling

In [6]:
# Sklearn HP search
# 1. 转化为 sklearn model
# 2. 定义参数集合
# 3. 搜索参数
def build_model(hidden_layers=1, layer_size=30, learning_rate=3e-3):
    model = keras.models.Sequential()
    model.add(keras.layers.Dense(layer_size, activation='relu', input_shape=xtrain.shape[1:]))
    for _ in range(hidden_layers - 1):
        model.add(keras.layers.Dense(layer_size, activation='relu'))

    model.add(keras.layers.Dense(1))
    optimizer = keras.optimizers.SGD(learning_rate)
    model.compile(loss="mse", optimizer=optimizer)
    return model


# callbacks = [keras.callbacks.EarlyStopping(patience=5, min_delta=1e-2)]
# sklearn_model = keras.wrappers.scikit_learn.KerasRegressor(build_model)
# history = sklearn_model.fit(xtrain_scaled, ytrain, 
#                             epochs=100, 
#                             validation_data=(xvalid_scaled, yvalid),
#                             callbacks=callbacks)

### Customized loss

In [10]:
## Custom loss
def customized_mse(y_true, y_pred):
    return tf.reduce_mean(tf.square(y_pred - y_true))

# Original model
model = keras.models.Sequential([
    keras.layers.Dense(30, activation='relu', input_shape=xtrain.shape[1:]),
    keras.layers.Dense(1),
])

model.summary()
# model.compile(loss="mean_squared_error", optimizer='sgd')
model.compile(loss=customized_mse, optimizer='sgd')
callbacks = [keras.callbacks.EarlyStopping(patience=5, min_delta=1e-5)]

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_4 (Dense)              (None, 30)                270       
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 31        
Total params: 301
Trainable params: 301
Non-trainable params: 0
_________________________________________________________________


### Customized layer

In [9]:
# tf.nn.softplus: log(1+e^x)  平滑版的 relu
customized_softplus = keras.layers.Lambda(lambda x: tf.nn.softplus(x))

class CustomizedDenseLayer(keras.layers.Layer):
    def __init__(self, units, activation=None, **kwargs):
        self.units = units  # number of nodes
        self.activation = keras.layers.Activation(activation)
        super(CustomizedDenseLayer, self).__init__(**kwargs)
    
    def build(self, input_shape):
        """Construct parameters"""
        self.kernel = self.add_weight(
            name="kernel", 
            shape=(input_shape[1], self.units),
            initializer="uniform",
            trainable=True)  # 在训练过程中可变
        self.bias = self.add_weight(
            name="bias",
            shape=(self.units,),
            initializer="zeros",
            trainable=True)  # 在训练过程中可变
        super(CustomizedDenseLayer, self).build(input_shape)

    ## call方法是自动被调用的，即如果你把这个类当做函数，后面写上括号，会自动调用 call 方法！
    def call(self, x):
        """正向计算，从输入到输出"""
        return self.activation(x@self.kernel + self.bias)


model = keras.models.Sequential([
    CustomizedDenseLayer(30, activation='relu', input_shape=xtrain.shape[1:]),
    CustomizedDenseLayer(1),
    customized_softplus
    # keras.layers.Dense(1, activation="softplus")
    # keras.layers.Dense(1), keras.layers.Activation("softplus")
])
model.summary()
model.compile(loss=customized_mse, optimizer='sgd', metrics=["mean_squared_error"])
callbacks = [keras.callbacks.EarlyStopping(patience=5, min_delta=1e-5)]

## Training

In [ ]:
history = model.fit(xtrain_scaled, ytrain,
                    validation_data=(xvalid_scaled, yvalid),
                    epochs=100,
                    callbacks=callbacks)

## Evaluation

In [ ]:
def plot_learning_curves(history):
    pd.DataFrame(history.history).plot(figsize=(8, 5))
    plt.grid(True)
    plt.gca().set_ylim(0, 1)
    plt.show()
    
plot_learning_curves(history)

In [ ]:
model.evaluate(xtest_scaled, ytest, verbose=0)

In [ ]:
from scipy.stats import reciprocal  # 定义搜索空间
# f(x) = 1/(x*log(b/a)) a <= x <= b

param_distribution = {
    "hidden_layers": [1, 2, 3, 4],
    "layer_size": np.arange(1, 100),
    "learning_rate": reciprocal(1e-4, 1e-2)
}

# RandomizedSearchCV
from sklearn.model_selection import RandomizedSearchCV
random_search_cv = RandomizedSearchCV(sklearn_model, param_distribution,
                                      n_iter=10, n_jobs=1)
random_search_cv.fit(xtrain_scaled, ytrain, epochs=100,
                     validation_data=(xvalid_scaled, yvalid),
                     callbacks=callbacks)

# 会使用 cross_validation

In [ ]:
print(random_search_cv.best_params_)
print(random_search_cv.best_score_)
print(random_search_cv.best_estimator_)

In [ ]:
model = random_search_cv.best_estimator_.model
model.evaluate(xtest_scaled, ytest)